<a href="https://colab.research.google.com/github/sokanaid/CourseProject2022BiologicalMath/blob/main/data_smoothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("hello")

hello
